In [1]:
from urllib.parse import urlparse
from collections import Counter
from tqdm import tqdm 
import pandas as pd
import pymysql, tldextract, json, os, ast

# Tracking Activity Analysis Notebook

In [2]:
summary_file = '../processed_data/tracking/sites_tracking_summary.csv'
summary_data = pd.read_csv(summary_file)
summary_data

category location                  etld  \
0       country_coded      UAE          www.tally.ae   
1       country_coded      UAE          www.tally.ae   
2       country_coded      UAE          www.tally.ae   
3       country_coded      UAE          www.tally.ae   
4       country_coded      UAE             porter.ae   
...               ...      ...                   ...   
246383         global      USA        saita-puls.com   
246384         global      USA        saita-puls.com   
246385         global      USA        saita-puls.com   
246386         global      USA  www.sciencealert.com   
246387         global      USA  www.sciencealert.com   

                                                      url  session_id  \
0         https://www.tally.ae/tally-prime-shortcuts.html           1   
1                                   https://www.tally.ae/           2   
2                           https://www.tally.ae/rewards/           3   
3                         https://www.tally.ae/index.html           4   
4                              https://porter.ae/about-us           5   
...                                                   ...         ...   
246383                   https://saita-puls.com/author/31      566322   
246384                  https://saita-puls.com/feature/54      566323   
246385                   https://saita-puls.com/author/84      566324   
246386  https://www.sciencealert.com/a-surprising-math...      566331   
246387  https://www.sciencealert.com/cannabis-compound...      566332   

        is_first_party_tracking  is_third_party_tracking  is_tracking  \
0                             0                        1            1   
1                             0                        1            1   
2                             0                        1            1   
3                             0                        1            1   
4                             0                        1            1   
...                         ...                      ...          ...   
246383                        0                        1            1   
246384                        0                        1            1   
246385                        0                        1            1   
246386                        0                        1            1   
246387                        0                        1            1   

                    distinct_third_party_tracking_domains  \
0       clarity.ms;doubleclick.net;google-analytics.co...   
1       clarity.ms;doubleclick.net;google-analytics.co...   
2       clarity.ms;doubleclick.net;google-analytics.co...   
3       clarity.ms;doubleclick.net;google-analytics.co...   
4       google-analytics.com;google.com;googletagmanag...   
...                                                   ...   
246383           google.com;googletagmanager.com;teads.tv   
246384           google.com;googletagmanager.com;teads.tv   
246385           google.com;googletagmanager.com;teads.tv   
246386  3lift.com;adnxs.com;adsrvr.org;amazon-adsystem...   
246387  1rx.io;3lift.com;acuityplatform.com;adform.net...   

                    distinct_third_party_tracking_origins  
0       analytics.google.com;googleads.g.doubleclick.n...  
1       analytics.google.com;googleads.g.doubleclick.n...  
2       analytics.google.com;googleads.g.doubleclick.n...  
3       analytics.google.com;googleads.g.doubleclick.n...  
4       analytics.google.com;o121817.ingest.us.sentry....  
...                                                   ...  
246383  a.teads.tv;analytics.google.com;at.teads.tv;ww...  
246384  a.teads.tv;analytics.google.com;at.teads.tv;ww...  
246385  a.teads.tv;analytics.google.com;at.teads.tv;ww...  
246386  6093eccf-6734-4877-ac8b-83d6d0e27b46.edge.perm...  
246387  6093eccf-6734-4877-ac8b-83d6d0e27b46.edge.perm...  

[246388 rows x 10 columns]

## General statistics

In [3]:
distinct_etlds = summary_data.groupby(['category','location'])['etld'].nunique().reset_index().rename(columns={'etld':'distinct_etlds'})
distinct_etlds

category location  distinct_etlds
0      country_coded  ALGERIA             206
1      country_coded  GERMANY            6181
2      country_coded    INDIA            5788
3      country_coded      UAE            1103
4   country_specific  ALGERIA            2434
5   country_specific  GERMANY            8460
6   country_specific    INDIA            8604
7   country_specific      UAE            8733
8             global  ALGERIA            4776
9             global  GERMANY            4776
10            global    INDIA            4776
11            global      UAE            4776
12            global      USA            4776

In [4]:
etld_url_counts = summary_data.groupby(['category','location','etld'])['url'].nunique().reset_index().rename(columns={'url':'n_unique_urls'})
agg_stats = etld_url_counts.groupby(['category','location'])['n_unique_urls'].agg(['mean','median','std','count']).reset_index()
agg_stats = agg_stats.rename(columns={'mean':'avg_urls_per_etld','median':'median_urls_per_etld','std':'std_urls_per_etld'})
agg_stats

category location  avg_urls_per_etld  median_urls_per_etld  \
0      country_coded  ALGERIA           4.291262                   5.0   
1      country_coded  GERMANY           4.590519                   5.0   
2      country_coded    INDIA           4.267623                   5.0   
3      country_coded      UAE           4.648232                   5.0   
4   country_specific  ALGERIA           3.773213                   5.0   
5   country_specific  GERMANY           4.262175                   5.0   
6   country_specific    INDIA           3.594375                   5.0   
7   country_specific      UAE           4.080385                   5.0   
8             global  ALGERIA           3.221524                   3.0   
9             global  GERMANY           3.546901                   4.0   
10            global    INDIA           1.809464                   1.0   
11            global      UAE           4.453727                   5.0   
12            global      USA           2.713358                   2.0   

    std_urls_per_etld  count  
0            1.421995    206  
1            2.673573   6181  
2            1.644195   5788  
3            1.951200   1103  
4            1.755833   2434  
5            2.234733   8460  
6            1.854913   8604  
7            1.583656   8733  
8            2.041128   4776  
9            3.225926   4776  
10           1.186716   4776  
11           1.739939   4776  
12           1.747554   4776

## Reading tracking requests

In [5]:
third_party_tracking_file = '../processed_data/tracking/third_party_tracking_requests.csv'
third_party_tracking_df = pd.read_csv(third_party_tracking_file)

print(f'Loaded {len(third_party_tracking_df)} third-party tracking requests')
third_party_tracking_df

Loaded 3465259 third-party tracking requests


category location                  etld  \
0        country_coded      UAE          www.tally.ae   
1        country_coded      UAE          www.tally.ae   
2        country_coded      UAE          www.tally.ae   
3        country_coded      UAE          www.tally.ae   
4        country_coded      UAE          www.tally.ae   
...                ...      ...                   ...   
3465254         global      USA  www.sciencealert.com   
3465255         global      USA  www.sciencealert.com   
3465256         global      USA  www.sciencealert.com   
3465257         global      USA  www.sciencealert.com   
3465258         global      USA  www.sciencealert.com   

                                                       url  session_id  \
0          https://www.tally.ae/tally-prime-shortcuts.html           1   
1          https://www.tally.ae/tally-prime-shortcuts.html           1   
2          https://www.tally.ae/tally-prime-shortcuts.html           1   
3          https://www.tally.ae/tally-prime-shortcuts.html           1   
4          https://www.tally.ae/tally-prime-shortcuts.html           1   
...                                                    ...         ...   
3465254  https://www.sciencealert.com/cannabis-compound...      566332   
3465255  https://www.sciencealert.com/cannabis-compound...      566332   
3465256  https://www.sciencealert.com/cannabis-compound...      566332   
3465257  https://www.sciencealert.com/cannabis-compound...      566332   
3465258  https://www.sciencealert.com/cannabis-compound...      566332   

                                          tracking_request  \
0        https://www.googletagmanager.com/gtm.js?id=GTM...   
1        https://www.googletagmanager.com/gtag/js?id=AW...   
2        https://www.googletagmanager.com/gtag/js?id=AW...   
3        https://www.googletagmanager.com/gtag/js?id=G-...   
4        https://www.googletagmanager.com/gtag/js?id=G-...   
...                                                    ...   
3465254  https://tpc.googlesyndication.com/pagead/js/r2...   
3465255  https://securepubads.g.doubleclick.net/pagead/...   
3465256  https://tpc.googlesyndication.com/pagead/js/r2...   
3465257  https://tpc.googlesyndication.com/pagead/js/r2...   
3465258  https://lg3-a.akamaihd.net/log?logid=kfk&evtid...   

        tracking_request_domain         tracking_request_origin  
0          googletagmanager.com        www.googletagmanager.com  
1          googletagmanager.com        www.googletagmanager.com  
2          googletagmanager.com        www.googletagmanager.com  
3          googletagmanager.com        www.googletagmanager.com  
4          googletagmanager.com        www.googletagmanager.com  
...                         ...                             ...  
3465254   googlesyndication.com       tpc.googlesyndication.com  
3465255         doubleclick.net  securepubads.g.doubleclick.net  
3465256   googlesyndication.com       tpc.googlesyndication.com  
3465257   googlesyndication.com       tpc.googlesyndication.com  
3465258            akamaihd.net              lg3-a.akamaihd.net  

[3465259 rows x 8 columns]

In [6]:
first_party_tracking_file = '../processed_data/tracking/first_party_tracking_requests.csv'
first_party_tracking_df = pd.read_csv(first_party_tracking_file)

print(f'Loaded {len(first_party_tracking_df)} first-party tracking requests')
first_party_tracking_df

Loaded 56854 first-party tracking requests


category location          etld  \
0      country_coded      UAE         24.ae   
1      country_coded      UAE       fmic.ae   
2      country_coded      UAE       fmic.ae   
3      country_coded      UAE       fmic.ae   
4      country_coded      UAE       fmic.ae   
...              ...      ...           ...   
56849         global      USA   khamsat.com   
56850         global      USA   khamsat.com   
56851         global      USA   khamsat.com   
56852         global      USA   khamsat.com   
56853         global      USA  clicks.co.za   

                                                     url  session_id  \
0                                         https://24.ae/          35   
1                                       https://fmic.ae/          92   
2      https://fmic.ae/intake-system/silicone-hoses/6...          93   
3                 https://fmic.ae/fuel-system/fuel-cells          94   
4          https://fmic.ae/brand/radium-engineering.html          95   
...                                                  ...         ...   
56849     https://khamsat.com/programming/ai-development      566260   
56850  https://khamsat.com/designing/social-media-design      566261   
56851  https://khamsat.com/designing/social-media-design      566261   
56852  https://khamsat.com/designing/social-media-design      566261   
56853  https://clicks.co.za/electrical/womens-groomin...      566305   

                                        tracking_request  \
0                             https://24.ae/cdn-cgi/rum?   
1                           https://fmic.ae/cdn-cgi/rum?   
2                           https://fmic.ae/cdn-cgi/rum?   
3                           https://fmic.ae/cdn-cgi/rum?   
4                           https://fmic.ae/cdn-cgi/rum?   
...                                                  ...   
56849  https://khamsat.com/ajax/account_stats?_=17624...   
56850  https://khamsat.com/ajax/account_stats?_=17624...   
56851  https://khamsat.com/ajax/account_stats?_=17624...   
56852  https://khamsat.com/ajax/account_stats?_=17624...   
56853                  https://clicks.co.za/cdn-cgi/rum?   

      tracking_request_domain tracking_request_origin  
0                       24.ae                   24.ae  
1                     fmic.ae                 fmic.ae  
2                     fmic.ae                 fmic.ae  
3                     fmic.ae                 fmic.ae  
4                     fmic.ae                 fmic.ae  
...                       ...                     ...  
56849             khamsat.com             khamsat.com  
56850             khamsat.com             khamsat.com  
56851             khamsat.com             khamsat.com  
56852             khamsat.com             khamsat.com  
56853            clicks.co.za            clicks.co.za  

[56854 rows x 8 columns]

## 1. Percentage of sites with tracking

In [7]:
# First: convert session-level summary to site-level summary
site_level = (
    summary_data
    .groupby(['category', 'location', 'etld'])
    .agg(
        site_has_tracking            = ('is_tracking', 'max'),
        site_has_first_party         = ('is_first_party_tracking', 'max'),
        site_has_third_party         = ('is_third_party_tracking', 'max'),
    )
    .reset_index()
)

# Now group by category + location at the site level
tracking_summary_sites = (
    site_level
    .groupby(['category', 'location'])
    .agg(
        s_sites = ('etld', 'nunique'),
        percent_tracking =
            ('site_has_tracking', lambda x: (x.sum() / x.count() * 100).round(2)),
        percent_first_party_tracking =
            ('site_has_first_party', lambda x: (x.sum() / x.count() * 100).round(2)),
        percent_third_party_tracking =
            ('site_has_third_party', lambda x: (x.sum() / x.count() * 100).round(2)),
    )
    .reset_index()
)

tracking_summary_sites


category location  s_sites  percent_tracking  \
0      country_coded  ALGERIA      206             67.48   
1      country_coded  GERMANY     6181             76.52   
2      country_coded    INDIA     5788             66.55   
3      country_coded      UAE     1103             83.68   
4   country_specific  ALGERIA     2434             45.36   
5   country_specific  GERMANY     8460             66.78   
6   country_specific    INDIA     8604             41.61   
7   country_specific      UAE     8733             73.11   
8             global  ALGERIA     4776             65.45   
9             global  GERMANY     4776             82.89   
10            global    INDIA     4776             74.08   
11            global      UAE     4776             72.95   
12            global      USA     4776             81.78   

    percent_first_party_tracking  percent_third_party_tracking  
0                           5.83                         67.48  
1                           6.58                         75.62  
2                           8.60                         65.67  
3                          12.33                         83.14  
4                           5.01                         44.17  
5                           4.43                         65.99  
6                           4.92                         41.38  
7                          12.20                         72.50  
8                           7.35                         65.20  
9                           8.61                         82.60  
10                          7.54                         73.74  
11                          6.18                         72.72  
12                         12.31                         81.41

In [8]:
# First: convert session-level summary to site-level summary
site_level = (
    summary_data
    .groupby(['category', 'location', 'etld'])
    .agg(
        site_has_tracking            = ('is_tracking', 'max'),
        site_has_first_party         = ('is_first_party_tracking', 'max'),
        site_has_third_party         = ('is_third_party_tracking', 'max'),
    )
    .reset_index()
)

# Now group by category + location at the site level
tracking_summary_sites = (
    site_level
    .groupby(['category', 'location'])
    .agg(
        s_sites = ('etld', 'nunique'),
        percent_tracking =
            ('site_has_tracking', lambda x: (x.sum() / x.count() * 100).round(2)),
        percent_first_party_tracking =
            ('site_has_first_party', lambda x: (x.sum() / x.count() * 100).round(2)),
        percent_third_party_tracking =
            ('site_has_third_party', lambda x: (x.sum() / x.count() * 100).round(2)),
    )
    .reset_index()
)

tracking_summary_sites


category location  s_sites  percent_tracking  \
0      country_coded  ALGERIA      206             67.48   
1      country_coded  GERMANY     6181             76.52   
2      country_coded    INDIA     5788             66.55   
3      country_coded      UAE     1103             83.68   
4   country_specific  ALGERIA     2434             45.36   
5   country_specific  GERMANY     8460             66.78   
6   country_specific    INDIA     8604             41.61   
7   country_specific      UAE     8733             73.11   
8             global  ALGERIA     4776             65.45   
9             global  GERMANY     4776             82.89   
10            global    INDIA     4776             74.08   
11            global      UAE     4776             72.95   
12            global      USA     4776             81.78   

    percent_first_party_tracking  percent_third_party_tracking  
0                           5.83                         67.48  
1                           6.58                         75.62  
2                           8.60                         65.67  
3                          12.33                         83.14  
4                           5.01                         44.17  
5                           4.43                         65.99  
6                           4.92                         41.38  
7                          12.20                         72.50  
8                           7.35                         65.20  
9                           8.61                         82.60  
10                          7.54                         73.74  
11                          6.18                         72.72  
12                         12.31                         81.41

## 2. Number of different third-party tracking domains 

In [9]:
def parse_domain_list(x):
    if pd.isna(x) or x.strip() == "":
        return []
    return [d.strip() for d in x.split(";") if d.strip() != ""]

In [10]:
summary_data["tp_domains_list"] = summary_data["distinct_third_party_tracking_domains"].apply(parse_domain_list)
summary_data["tp_origins_list"] = summary_data["distinct_third_party_tracking_origins"].apply(parse_domain_list)


In [11]:

site_level = (
    summary_data.groupby(["category", "location", "etld"])
    .agg(
        # unique TP domains seen for this site
        site_tp_domains=("tp_domains_list", lambda lsts: set(d for s in lsts for d in s)),
        
        # unique TP origins seen for this site
        site_tp_origins=("tp_origins_list", lambda lsts: set(o for s in lsts for o in s)),
    )
    .reset_index()
)

site_level["num_tp_domains"] = site_level["site_tp_domains"].apply(len)

result = (
    site_level.groupby(["category", "location"])
    .agg(
        # stats for the number of third-party domains PER ETLD
        avg_third_party_domains_per_etld=("num_tp_domains", "mean"),
        median_third_party_domains_per_etld=("num_tp_domains", "median"),
        std_third_party_domains_per_etld=("num_tp_domains", "std"),
        min_third_party_domains_per_etld=("num_tp_domains", "min"),
        max_third_party_domains_per_etld=("num_tp_domains", "max"),

        # union of all origins in this category/location
        all_tp_origins=("site_tp_origins", lambda sets: set.union(*sets) if len(sets) > 0 else set()),
    )
    .reset_index()
)

# Count distinct origins
result["number_distinct_third_party_tracking_origins"] = result["all_tp_origins"].apply(len)

# Clean up
result = result.drop(columns=["all_tp_origins"])
result = result.sort_values(["category", "location"]).reset_index(drop=True)

# Show final table
result


category location  avg_third_party_domains_per_etld  \
0      country_coded  ALGERIA                          2.859223   
1      country_coded  GERMANY                          5.952435   
2      country_coded    INDIA                          3.661023   
3      country_coded      UAE                          6.766092   
4   country_specific  ALGERIA                          1.263763   
5   country_specific  GERMANY                          5.356383   
6   country_specific    INDIA                          2.109716   
7   country_specific      UAE                          4.025306   
8             global  ALGERIA                          9.015494   
9             global  GERMANY                         11.977387   
10            global    INDIA                          6.531407   
11            global      UAE                         14.036013   
12            global      USA                         10.692420   

    median_third_party_domains_per_etld  std_third_party_domains_per_etld  \
0                                   2.0                          3.538580   
1                                   3.0                          8.232798   
2                                   2.0                          5.337170   
3                                   5.0                          7.711309   
4                                   0.0                          2.240077   
5                                   2.0                         11.119543   
6                                   0.0                          4.116665   
7                                   3.0                          4.583543   
8                                   3.0                         14.573912   
9                                   5.0                         16.353885   
10                                  3.0                          8.827839   
11                                  5.0                         20.054918   
12                                  5.0                         14.457123   

    min_third_party_domains_per_etld  max_third_party_domains_per_etld  \
0                                  0                                27   
1                                  0                                89   
2                                  0                                58   
3                                  0                                75   
4                                  0                                32   
5                                  0                                91   
6                                  0                                57   
7                                  0                                76   
8                                  0                                87   
9                                  0                               100   
10                                 0                                72   
11                                 0                               116   
12                                 0                               107   

    number_distinct_third_party_tracking_origins  
0                                            131  
1                                           3795  
2                                           1777  
3                                            881  
4                                            356  
5                                           4267  
6                                           1214  
7                                           1590  
8                                           4539  
9                                           5554  
10                                          4032  
11                                          5343  
12                                          5173

# 3. Most common third-party tracking domains

In [12]:
columns_data = {}

# Group by category and location
for (cat, loc), group in summary_data.groupby(['category', 'location']):
    total_sites = group.shape[0]
    
    # Count domains
    domain_counts = Counter()

    for domains_str in group['distinct_third_party_tracking_origins']:

        if pd.isna(domains_str):
            continue

        domains = set(d.strip() for d in domains_str.split(';') if d.strip())

        for domain in domains:
            domain_counts[domain] += 1
    
    # Compute percentage
    domain_percentages = {domain: count / total_sites * 100 for domain, count in domain_counts.items()}
    
    # Sort by percentage descending
    sorted_domains = sorted(domain_percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Format as "domain (xx%)"
    formatted_domains = [f"{domain} ({perc:.1f}%)" for domain, perc in sorted_domains]
    
    # Store in dict
    columns_data[f"{cat}-{loc}"] = formatted_domains

# Find max number of domains across all columns for consistent row length
max_len = max(len(domains) for domains in columns_data.values())

# Pad shorter columns with empty strings
for col, domains in columns_data.items():
    if len(domains) < max_len:
        domains.extend([''] * (max_len - len(domains)))

# Build DataFrame
df_top_domains = pd.DataFrame(columns_data)

df_top_domains.head(20)

country_coded-ALGERIA  \
0       www.googletagmanager.com (45.0%)   
1       www.google-analytics.com (33.2%)   
2           connect.facebook.net (18.8%)   
3               www.facebook.com (17.2%)   
4            maps.googleapis.com (12.3%)   
5           analytics.google.com (10.8%)   
6                 www.google.com (10.1%)   
7     googleads.g.doubleclick.net (5.8%)   
8                   www.google.dz (4.5%)   
9                 s.go-mpulse.net (4.1%)   
10  pagead2.googlesyndication.com (3.5%)   
11                c.go-mpulse.net (3.1%)   
12         static.doubleclick.net (2.6%)   
13                play.google.com (2.6%)   
14                www.youtube.com (2.6%)   
15                 www.clarity.ms (2.5%)   
16             ad.doubleclick.net (2.5%)   
17        stats.g.doubleclick.net (2.4%)   
18                 snap.licdn.com (2.4%)   
19            px.ads.linkedin.com (2.4%)   

                    country_coded-GERMANY  \
0        www.googletagmanager.com (55.3%)   
1                  www.google.com (29.9%)   
2    region1.google-analytics.com (27.4%)   
3            connect.facebook.net (24.0%)   
4                www.facebook.com (22.6%)   
5     googleads.g.doubleclick.net (21.1%)   
6    region1.analytics.google.com (20.7%)   
7        www.google-analytics.com (15.2%)   
8   pagead2.googlesyndication.com (14.1%)   
9                    bat.bing.com (12.8%)   
10                  www.google.de (12.4%)   
11                  www.clarity.ms (9.1%)   
12            imgsct.cookiebot.com (6.9%)   
13         stats.g.doubleclick.net (6.5%)   
14            analytics.tiktok.com (4.8%)   
15                    bat.bing.net (4.6%)   
16                  snap.licdn.com (4.0%)   
17              ad.doubleclick.net (3.9%)   
18                    s.pinimg.com (3.8%)   
19             px.ads.linkedin.com (3.8%)   

                          country_coded-INDIA  \
0            www.googletagmanager.com (44.7%)   
1            www.google-analytics.com (35.6%)   
2                      www.google.com (17.5%)   
3         googleads.g.doubleclick.net (17.2%)   
4                analytics.google.com (13.5%)   
5                connect.facebook.net (12.0%)   
6                    www.facebook.com (11.6%)   
7        pagead2.googlesyndication.com (9.3%)   
8                     www.google.co.in (6.6%)   
9                       www.clarity.ms (6.2%)   
10                 maps.googleapis.com (6.2%)   
11        monorail-edge.shopifysvc.com (4.0%)   
12       static.cloudflareinsights.com (3.2%)   
13                     play.google.com (3.1%)   
14              static.doubleclick.net (3.0%)   
15                     www.youtube.com (2.9%)   
16             stats.g.doubleclick.net (2.9%)   
17                        stats.wp.com (2.6%)   
18  www.merchant-center-analytics.goog (2.6%)   
19                        pixel.wp.com (2.5%)   

                            country_coded-UAE  \
0            www.googletagmanager.com (69.3%)   
1            www.google-analytics.com (46.3%)   
2                      www.google.com (42.8%)   
3         googleads.g.doubleclick.net (37.4%)   
4                analytics.google.com (32.6%)   
5                connect.facebook.net (29.6%)   
6                    www.facebook.com (27.7%)   
7                       www.google.ae (14.3%)   
8                      www.clarity.ms (11.5%)   
9                analytics.tiktok.com (11.2%)   
10                       sc-static.net (9.4%)   
11        monorail-edge.shopifysvc.com (9.0%)   
12                     tr.snapchat.com (8.9%)   
13                 maps.googleapis.com (8.5%)   
14                 px.ads.linkedin.com (7.9%)   
15             stats.g.doubleclick.net (7.6%)   
16                      snap.licdn.com (7.5%)   
17       pagead2.googlesyndication.com (7.4%)   
18  www.merchant-center-analytics.goog (6.8%)   
19                        bat.bing.com (6.3%)   

                country_specific-ALGERIA  \
0       www.googletagmanager.com (22.6%)   


In [13]:
columns_data = {}

# Group only by location
for loc, group in summary_data.groupby('location'):
    total_sites = group.shape[0]
    
    # Count domains
    domain_counts = Counter()
    for domains_str in group['distinct_third_party_tracking_origins']:
        if pd.isna(domains_str):
            continue

        # Split semicolon-separated domains and remove duplicates per site
        domains = set(d.strip() for d in domains_str.split(';') if d.strip())
        for domain in domains:
            domain_counts[domain] += 1

    # Compute percentage of sites
    domain_percentages = {domain: count / total_sites * 100 for domain, count in domain_counts.items()}

    # Sort descending
    sorted_domains = sorted(domain_percentages.items(), key=lambda x: x[1], reverse=True)

    # Format as "domain (xx%)"
    formatted_domains = [f"{domain} ({perc:.1f}%)" for domain, perc in sorted_domains]

    columns_data[loc] = formatted_domains

# Pad columns to equal length
max_len = max(len(domains) for domains in columns_data.values())
for domains in columns_data.values():
    domains.extend([''] * (max_len - len(domains)))

# Build DataFrame
df_top_domains_by_location = pd.DataFrame(columns_data)

df_top_domains_by_location.head(20)

ALGERIA  \
0        www.googletagmanager.com (38.0%)   
1        www.google-analytics.com (25.6%)   
2   pagead2.googlesyndication.com (13.8%)   
3            analytics.google.com (13.4%)   
4                  www.google.com (10.8%)   
5            connect.facebook.net (10.0%)   
6   securepubads.g.doubleclick.net (9.9%)   
7      googleads.g.doubleclick.net (8.5%)   
8                 www.facebook.com (8.1%)   
9                    www.google.dz (7.9%)   
10   static.cloudflareinsights.com (6.8%)   
11                    ib.adnxs.com (5.0%)   
12         stats.g.doubleclick.net (4.8%)   
13             maps.googleapis.com (4.5%)   
14                    id5-sync.com (4.2%)   
15           c.amazon-adsystem.com (4.2%)   
16            cm.g.doubleclick.net (3.9%)   
17                match.adsrvr.org (3.9%)   
18                  www.clarity.ms (3.8%)   
19        pixel.rubiconproject.com (3.5%)   

                                  GERMANY  \
0        www.googletagmanager.com (49.0%)   
1    region1.google-analytics.com (24.8%)   
2                  www.google.com (23.5%)   
3    region1.analytics.google.com (18.4%)   
4            connect.facebook.net (18.0%)   
5                www.facebook.com (17.0%)   
6   pagead2.googlesyndication.com (15.9%)   
7     googleads.g.doubleclick.net (15.9%)   
8        www.google-analytics.com (13.6%)   
9                   www.google.de (10.9%)   
10                    bat.bing.com (8.9%)   
11  securepubads.g.doubleclick.net (7.7%)   
12                  www.clarity.ms (6.9%)   
13         stats.g.doubleclick.net (6.0%)   
14            cm.g.doubleclick.net (5.3%)   
15            imgsct.cookiebot.com (4.7%)   
16       tpc.googlesyndication.com (4.6%)   
17                    ib.adnxs.com (4.4%)   
18   static.cloudflareinsights.com (4.3%)   
19                 x.bidswitch.net (4.0%)   

                                        INDIA  \
0            www.googletagmanager.com (39.4%)   
1            www.google-analytics.com (30.3%)   
2                      www.google.com (13.8%)   
3         googleads.g.doubleclick.net (13.1%)   
4                analytics.google.com (12.7%)   
5        pagead2.googlesyndication.com (9.9%)   
6                 connect.facebook.net (9.6%)   
7                     www.facebook.com (9.0%)   
8                     www.google.co.in (7.5%)   
9                       www.clarity.ms (5.3%)   
10       static.cloudflareinsights.com (4.0%)   
11                 maps.googleapis.com (3.9%)   
12      securepubads.g.doubleclick.net (3.3%)   
13             stats.g.doubleclick.net (3.3%)   
14        monorail-edge.shopifysvc.com (3.0%)   
15           tpc.googlesyndication.com (2.4%)   
16                cm.g.doubleclick.net (2.3%)   
17  www.merchant-center-analytics.goog (2.1%)   
18                        stats.wp.com (2.0%)   
19                        pixel.wp.com (1.9%)   

                                      UAE  \
0        www.googletagmanager.com (60.7%)   
1        www.google-analytics.com (41.7%)   
2                  www.google.com (33.1%)   
3     googleads.g.doubleclick.net (29.0%)   
4            analytics.google.com (23.9%)   
5            connect.facebook.net (21.4%)   
6                www.facebook.com (20.1%)   
7   pagead2.googlesyndication.com (12.1%)   
8                   www.google.ae (10.3%)   
9                   www.clarity.ms (9.7%)   
10  securepubads.g.doubleclick.net (8.4%)   
11             maps.googleapis.com (7.4%)   
12            analytics.tiktok.com (5.6%)   
13       tpc.googlesyndication.com (5.6%)   
14            cm.g.doubleclick.net (5.5%)   
15         stats.g.doubleclick.net (5.3%)   
16   static.cloudflareinsights.com (5.2%)   
17                    ib.adnxs.com (4.6%)   
18             px.ads.linkedin.com (4.4%)   
19    monorail-edge.shopifysvc.com (3.9%)   

                                       USA  
0         www.googletagmanager.com (56.8%)  
1         www.google-analytics.com (37.2%)  
2             analytics.google.

In [14]:
columns_data = {}

# Group only by location
for loc, group in summary_data.groupby('location'):
    total_sites = group.shape[0]
    
    # Count domains
    domain_counts = Counter()
    for domains_str in group['distinct_third_party_tracking_domains']:
        if pd.isna(domains_str):
            continue

        # Split semicolon-separated domains and remove duplicates per site
        domains = set(d.strip() for d in domains_str.split(';') if d.strip())
        for domain in domains:
            domain_counts[domain] += 1

    # Compute percentage of sites
    domain_percentages = {domain: count / total_sites * 100 for domain, count in domain_counts.items()}

    # Sort descending
    sorted_domains = sorted(domain_percentages.items(), key=lambda x: x[1], reverse=True)

    # Format as "domain (xx%)"
    formatted_domains = [f"{domain} ({perc:.1f}%)" for domain, perc in sorted_domains]

    columns_data[loc] = formatted_domains

# Pad columns to equal length
max_len = max(len(domains) for domains in columns_data.values())
for domains in columns_data.values():
    domains.extend([''] * (max_len - len(domains)))

# Build DataFrame
df_top_domains_by_location = pd.DataFrame(columns_data)

df_top_domains_by_location.head(20)

ALGERIA                        GERMANY  \
0    googletagmanager.com (38.0%)   googletagmanager.com (49.0%)   
1    google-analytics.com (26.7%)   google-analytics.com (31.2%)   
2              google.com (19.9%)             google.com (30.8%)   
3         doubleclick.net (18.6%)        doubleclick.net (25.9%)   
4   googlesyndication.com (13.8%)           facebook.net (18.0%)   
5            facebook.net (10.0%)           facebook.com (17.0%)   
6             facebook.com (8.1%)  googlesyndication.com (15.9%)   
7                google.dz (7.9%)              google.de (10.9%)   
8   cloudflareinsights.com (6.8%)                bing.com (8.9%)   
9           googleapis.com (6.5%)              clarity.ms (6.9%)   
10      rubiconproject.com (5.5%)               adnxs.com (5.2%)   
11               adnxs.com (5.5%)            pubmatic.com (4.9%)   
12            pubmatic.com (5.3%)           cookiebot.com (4.7%)   
13     amazon-adsystem.com (4.7%)      rubiconproject.com (4.6%)   
14            id5-sync.com (4.6%)              adform.net (4.4%)   
15         creativecdn.com (4.2%)  cloudflareinsights.com (4.3%)   
16              adsrvr.org (3.9%)            id5-sync.com (4.3%)   
17              clarity.ms (3.9%)     amazon-adsystem.com (4.3%)   
18         casalemedia.com (3.8%)          googleapis.com (4.1%)   
19   scorecardresearch.com (3.5%)           bidswitch.net (4.1%)   

                                    INDIA                            UAE  \
0            googletagmanager.com (39.4%)   googletagmanager.com (60.7%)   
1            google-analytics.com (30.8%)   google-analytics.com (42.2%)   
2                      google.com (21.0%)             google.com (41.8%)   
3                 doubleclick.net (17.7%)        doubleclick.net (36.8%)   
4            googlesyndication.com (9.9%)           facebook.net (21.4%)   
5                     facebook.net (9.6%)           facebook.com (20.1%)   
6                     facebook.com (9.0%)  googlesyndication.com (12.1%)   
7                     google.co.in (7.5%)              google.ae (10.3%)   
8                       clarity.ms (5.3%)              clarity.ms (9.7%)   
9                   googleapis.com (4.6%)          googleapis.com (9.4%)   
10          cloudflareinsights.com (4.0%)              tiktok.com (5.6%)   
11                  shopifysvc.com (3.1%)      rubiconproject.com (5.3%)   
12  merchant-center-analytics.goog (2.1%)  cloudflareinsights.com (5.2%)   
13                          wp.com (2.0%)               adnxs.com (5.1%)   
14                    linkedin.com (1.7%)            pubmatic.com (4.9%)   
15                        bing.com (1.6%)            linkedin.com (4.4%)   
16                       licdn.com (1.5%)     amazon-adsystem.com (4.3%)   
17                     youtube.com (1.5%)          shopifysvc.com (3.9%)   
18                       adnxs.com (1.4%)              adform.net (3.9%)   
19              rubiconproject.com (1.4%)         creativecdn.com (3.9%)   

                               USA  
0     googletagmanager.com (56.8%)  
1     google-analytics.com (37.8%)  
2               google.com (33.2%)  
3          doubleclick.net (31.8%)  
4    googlesyndication.com (21.5%)  
5   cloudflareinsights.com (21.0%)  
6             facebook.net (14.2%)  
7             facebook.com (12.8%)  
8                adnxs.com (10.1%)  
9        rubiconproject.com (8.9%)  
10             pubmatic.com (8.7%)  
11          creativecdn.com (8.1%)  
12          casalemedia.com (7.5%)  
13      amazon-adsystem.com (7.1%)  
14               clarity.ms (7.0%)  
15             id5-sync.com (6.6%)  
16            bidswitch.net (6.4%)  
17        smartadserver.com (6.1%)  
18                openx.net (6.0%)  
19                 teads.tv (6.0%)

# 4. Most common tracking organizations

In [15]:
# Load the DuckDuckGo domain map
with open("utils/duckduckgo_domain_map.json") as f:
    radar_data = json.load(f)

def get_organization(domain):
    info = radar_data.get(domain) or {}
    return info.get('displayName', 'Unknown')


# Dictionary to hold top organizations per category-location
columns_data = {}

# Group by category and location
for (cat, loc), group in summary_data.groupby(['category', 'location']):
    total_sites = group.shape[0]
    
    # Count organizations
    org_counts = Counter()
    for domains_str in group['distinct_third_party_tracking_domains']:

        if pd.isna(domains_str):
            continue

        domains = set(d.strip() for d in domains_str.split(';') if d.strip())
        # Get unique organizations per site
        organizations = {get_organization(domain) for domain in domains}
        for org in organizations:
            org_counts[org] += 1
    
    # Compute percentage
    org_percentages = {org: count / total_sites * 100 for org, count in org_counts.items()}
    
    # Sort by percentage descending
    sorted_orgs = sorted(org_percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Format as "organization (xx%)"
    formatted_orgs = [f"{org} ({perc:.1f}%)" for org, perc in sorted_orgs]
    
    # Store in dict
    columns_data[f"{cat}-{loc}"] = formatted_orgs

# Find max number of orgs across all columns for consistent row length
max_len = max(len(orgs) for orgs in columns_data.values())

# Pad shorter columns with empty strings
for col, orgs in columns_data.items():
    if len(orgs) < max_len:
        orgs.extend([''] * (max_len - len(orgs)))

# Build DataFrame
df_top_organizations = pd.DataFrame(columns_data)

df_top_organizations.head(20)


country_coded-ALGERIA country_coded-GERMANY    country_coded-INDIA  \
0         Google (52.7%)        Google (60.9%)         Google (53.0%)   
1       Facebook (18.9%)       Unknown (31.1%)        Unknown (14.8%)   
2        Unknown (13.8%)      Facebook (24.5%)       Facebook (12.2%)   
3       Microsoft (5.2%)     Microsoft (21.3%)       Microsoft (9.4%)   
4          Akamai (4.1%)   Usercentrics (7.8%)         Shopify (4.0%)   
5           Adobe (2.0%)          Adobe (5.5%)      Cloudflare (3.4%)   
6           Teads (1.9%)      ByteDance (5.0%)      Automattic (2.6%)   
7      Automattic (1.7%)     Amazon.com (4.5%)           Adobe (2.2%)   
8       New Relic (1.6%)      Pinterest (4.1%)      Amazon.com (1.9%)   
9         Twitter (1.5%)        Klaviyo (3.5%)         GoDaddy (1.3%)   
10    AT Internet (1.5%)     Cloudflare (3.1%)          Akamai (1.3%)   
11       OneTrust (1.2%)  Virtual Minds (2.8%)       OneSignal (1.1%)   
12     Cloudflare (1.1%)        Shopify (2.7%)       Starfield (1.0%)   
13        Shopify (1.0%)     Trustpilot (2.6%)         Twitter (0.9%)   
14        Renault (0.7%)          Teads (2.5%)         IPONWEB (0.8%)   
15      ByteDance (0.7%)         Adform (2.5%)       RTB House (0.8%)   
16           Snap (0.6%)        IPONWEB (2.4%)       ShareThis (0.8%)   
17     Trustpilot (0.6%)           Awin (2.4%)           Teads (0.8%)   
18         Reddit (0.6%)       PubMatic (2.4%)  Index Exchange (0.8%)   
19     Amazon.com (0.6%)        Magnite (2.4%)          Adform (0.7%)   

        country_coded-UAE country_specific-ALGERIA country_specific-GERMANY  \
0          Google (73.2%)           Google (35.3%)           Google (46.6%)   
1         Unknown (31.2%)           Unknown (7.2%)          Unknown (30.2%)   
2        Facebook (29.9%)          Facebook (4.9%)        Microsoft (13.7%)   
3       Microsoft (22.1%)        Automattic (3.8%)         Facebook (13.6%)   
4       ByteDance (11.6%)        Cloudflare (1.1%)      Usercentrics (5.9%)   
5             Snap (9.5%)           Twitter (0.8%)        Amazon.com (5.1%)   
6          Shopify (9.0%)         Microsoft (0.7%)     Virtual Minds (4.6%)   
7            Adobe (5.2%)        Amazon.com (0.4%)             Adobe (4.6%)   
8       Cloudflare (4.6%)         OneSignal (0.4%)            Adform (4.6%)   
9   Salesforce.com (3.8%)         ByteDance (0.3%)          PubMatic (4.0%)   
10          Yandex (3.6%)             Adobe (0.3%)           IPONWEB (3.9%)   
11         Twitter (3.6%)           HubSpot (0.3%)               ID5 (3.6%)   
12      Amazon.com (2.7%)         ShareThis (0.2%)           Magnite (3.3%)   
13         Klaviyo (2.7%)           Shopify (0.2%)     Smartadserver (3.3%)   
14       New Relic (2.6%)         Pinterest (0.2%)        TripleLift (2.9%)   
15           Teads (2.5%)    Sovrn Holdings (0.2%)         ByteDance (2.9%)   
16          Akamai (2.3%)            Disqus (0.1%)    Index Exchange (2.8%)   
17         HubSpot (2.2%)        LiveIntent (0.1%)         Pinterest (2.7%)   
18        PubMatic (2.0%)       Zeta Global (0.1%)             Teads (2.7%)   
19         IPONWEB (1.9%)           Bombora (0.1%)        Cloudflare (2.5%)   

            country_specific-INDIA country_specific-UAE  \
0                   Google (35.8%)       Google (66.2%)   
1                   Unknown (8.7%)     Facebook (22.1%)   
2                  Facebook (7.0%)      Unknown (20.5%)   
3                 Microsoft (5.7%)    Microsoft (13.7%)   
4                   Shopify (3.2%)       Shopify (5.5%)   
5                Cloudflare (2.9%)     ByteDance (5.0%)   
6                Automattic (1.8%)          Snap (3.9%)   
7                 OneSignal (1.1%)    Cloudflare (3.7%)   
8                Amazon.com (1.0%)    Automattic (2.7%)   
9                   Taboola (0.9%)        Yandex (2.4%)   
10                 comScore (0.8%)         Adobe (1.7%)   
11                RTB House (0.7%)       Twitter (1.6%)   
12  Exponential Interactive (0.6%)       G

# 5. Country-specific tracking domains

In [16]:
df_tracking = summary_data[
    summary_data['is_third_party_tracking'] == 1
].copy()

countries = df_tracking['location'].unique()
table_dict = {country: [] for country in countries}

for country in countries:

    df_country = df_tracking[df_tracking['location'] == country]
    df_other = df_tracking[df_tracking['location'] != country]

    # Collect all third-party origins in other countries
    origins_other = set()
    for s in df_other['distinct_third_party_tracking_domains']:
        if pd.isna(s):
            continue
        origins_other.update(d.strip() for d in s.split(';') if d.strip())

    # Collect origins unique to this country
    origins_country_only = []
    for s in df_country['distinct_third_party_tracking_domains']:
        if pd.isna(s):
            continue
        domains = set(d.strip() for d in s.split(';') if d.strip())
        for domain in domains:
            if domain not in origins_other:
                origins_country_only.append(domain)

    # Count frequency
    counter = Counter(origins_country_only)
    n_sites = len(df_country)

    table_dict[country] = [
        {
            "domain": domain,
            "pct_sites": count / n_sites * 100
        }
        for domain, count in counter.most_common(20)
    ]

    print(country, len(counter))

# Build DataFrame (pad uneven columns)
max_len = max(len(v) for v in table_dict.values())

df_result = pd.DataFrame({
    country: [
        f"{table_dict[country][i]['domain']} ({table_dict[country][i]['pct_sites']:.1f}%)"
        if i < len(table_dict[country]) else ""
        for i in range(max_len)
    ]
    for country in countries
})

df_result

UAE 1000
ALGERIA 188
GERMANY 4201
INDIA 892
USA 171


UAE                   ALGERIA  \
0               google.ae (14.1%)         google.dz (14.3%)   
1                yads.tech (0.7%)     univ-batna2.dz (0.9%)   
2             tildacdn.one (0.3%)         naltis.com (0.2%)   
3                  seha.ae (0.1%)     7345dc0372.com (0.2%)   
4            contentcdn.ae (0.1%)             esi.dz (0.2%)   
5             clixtell.com (0.1%)          ezmob.com (0.2%)   
6           flyingtiger.ae (0.1%)     91842100df.com (0.1%)   
7         dubailand.gov.ae (0.1%)     univ-annaba.dz (0.1%)   
8             iconnode.com (0.1%)     univ-eloued.dz (0.1%)   
9             tildacdn.net (0.1%)     d6468dab54.com (0.1%)   
10          a3c72512d5.com (0.1%)  univ-adrar.edu.dz (0.1%)   
11               amazon.ae (0.1%)     e55a2001b9.com (0.1%)   
12             chatfood.io (0.0%)      hotelhydra.dz (0.1%)   
13             bitrix24.ru (0.0%)      midocamera.dz (0.1%)   
14          8eabd9b030.com (0.0%)     43201617eb.com (0.1%)   
15               amocrm.ru (0.0%)           estin.dz (0.1%)   
16  ellingtonproperties.ae (0.0%)        univ-sba.dz (0.1%)   
17             adek.gov.ae (0.0%)    radioalgerie.dz (0.1%)   
18          sitefinity.com (0.0%)         renault.dz (0.0%)   
19          70de8b3bd7.com (0.0%)      bali6nora.com (0.0%)   

                    GERMANY                      INDIA  \
0         google.de (15.2%)       google.co.in (14.6%)   
1            ioam.de (0.7%)       samarth.ac.in (0.3%)   
2           hyj.mobi (0.6%)  stripchatgirls.com (0.2%)   
3         adcell.com (0.4%)           amazon.in (0.2%)   
4         benelph.de (0.4%)      superbot.works (0.1%)   
5         ecn-ldr.de (0.4%)            fynd.com (0.1%)   
6        gsitrix.com (0.3%)      1cd21cb1e4.com (0.1%)   
7    spoteffects.net (0.3%)      c6073077cc.com (0.1%)   
8    helloretail.com (0.2%)            dotpe.in (0.1%)   
9          bf-ad.net (0.2%)         tosshub.com (0.1%)   
10   funkedigital.de (0.2%)      dotshowroom.in (0.1%)   
11      sparkasse.de (0.2%)  paupsoborofoow.net (0.1%)   
12  klarinsights.net (0.2%)    blitzshopdeck.in (0.1%)   
13         kctag.net (0.2%)        symptoma.com (0.0%)   
14    372ae6b830.com (0.1%)          vizury.com (0.0%)   
15       uimserv.net (0.1%)   chaiserepiece.com (0.0%)   
16            nrw.de (0.1%)       groleegni.net (0.0%)   
17       advantic.de (0.1%)      c84c10fe3d.com (0.0%)   
18       adspirit.de (0.1%)        vaugroar.com (0.0%)   
19    lhinsights.com (0.1%)    vivastreet.co.in (0.0%)   

                             USA  
0              tpmn.co.kr (1.3%)  
1          6e64d84de7.com (0.2%)  
2          d6f9e3ab62.com (0.2%)  
3              pippio.com (0.2%)  
4          2e68bd0a8b.com (0.1%)  
5        geniusmonkey.com (0.1%)  
6            qwerty24.net (0.1%)  
7               beeg.porn (0.1%)  
8          d4020f5f0f.com (0.1%)  
9              heeporn.me (0.1%)  
10            naasongs.co (0.0%)  
11            porndos.com (0.0%)  
12             celtra.com (0.0%)  
13  camshowrecordings.com (0.0%)  
14              xxxi.porn (0.0%)  
15          keledushu.com (0.0%)  
16               klart.se (0.0%)  
17         ejetplay.click (0.0%)  
18          soccer6.co.za (0.0%)  
19                 com.se (0.0%)

In [17]:

# Filter only rows with third-party tracking (FIXED)
df_tracking = summary_data[
    summary_data['is_third_party_tracking'] == 1
].copy()

countries = df_tracking['location'].unique()
table_dict = {country: [] for country in countries}

for country in countries:

    df_country = df_tracking[df_tracking['location'] == country]
    df_other = df_tracking[df_tracking['location'] != country]

    # Collect all third-party origins in other countries
    origins_other = set()
    for s in df_other['distinct_third_party_tracking_origins']:
        if pd.isna(s):
            continue
        origins_other.update(o.strip() for o in s.split(';') if o.strip())

    # Collect origins unique to this country
    origins_country_only = []
    for s in df_country['distinct_third_party_tracking_origins']:
        if pd.isna(s):
            continue
        origins = set(o.strip() for o in s.split(';') if o.strip())
        for origin in origins:
            if origin not in origins_other:
                origins_country_only.append(origin)

    # Count frequency
    counter = Counter(origins_country_only)
    n_sites = len(df_country)

    table_dict[country] = [
        {
            "origin": origin,
            "pct_sites": count / n_sites * 100
        }
        for origin, count in counter.most_common(30)
    ]

    print(country, len(counter))

# ---- Build DataFrame safely (pad uneven columns) ----
max_len = max(len(v) for v in table_dict.values())

df_result = pd.DataFrame({
    country: (
        [f"{e['origin']} ({e['pct_sites']:.1f}%)" for e in table_dict[country]]
        + [""] * (max_len - len(table_dict[country]))
    )
    for country in countries
})

df_result


UAE 1815
ALGERIA 622
GERMANY 6140
INDIA 1550
USA 628


UAE  \
0                               www.google.ae (14.1%)   
1                            display.yads.tech (0.7%)   
2                          static.tildacdn.one (0.3%)   
3                         ittr-asia.eskimi.com (0.1%)   
4                                contentcdn.ae (0.1%)   
5                         scripts.clixtell.com (0.1%)   
6                           www.flyingtiger.ae (0.1%)   
7                         process.iconnode.com (0.1%)   
8                             dubailand.gov.ae (0.1%)   
9                          static.tildacdn.net (0.1%)   
10                   cad4defd63.a3c72512d5.com (0.1%)   
11  wix-engage-visitors-prod-11.firebaseio.com (0.1%)   
12                             unagi.amazon.ae (0.1%)   
13                            ampcid.google.ae (0.1%)   
14  wix-engage-visitors-prod-10.firebaseio.com (0.1%)   
15                 o141485.ingest.us.sentry.io (0.0%)   
16                           order.chatfood.io (0.0%)   
17                          widget.chatfood.io (0.0%)   
18                          cdn-ru.bitrix24.ru (0.0%)   
19                   c34352e2b2.8eabd9b030.com (0.0%)   
20  wix-engage-visitors-prod-25.firebaseio.com (0.0%)   
21                             piper.amocrm.ru (0.0%)   
22                               www.amazon.ae (0.0%)   
23                            fls-eu.amazon.ae (0.0%)   
24                  s2s.ellingtonproperties.ae (0.0%)   
25               nl-api.insight.sitefinity.com (0.0%)   
26                   7b1deec9a7.70de8b3bd7.com (0.0%)   
27                         sentry.wixpress.com (0.0%)   
28  wix-engage-visitors-prod-19.firebaseio.com (0.0%)   
29                               cdn2.teads.tv (0.0%)   

                                    ALGERIA  \
0                     www.google.dz (14.3%)   
1                     nws.naltis.com (0.2%)   
2          dbb8b55e45.7345dc0372.com (0.2%)   
3            rtb-useast-v4.ezmob.com (0.2%)   
4                   ampcid.google.dz (0.2%)   
5   uvi-12796.agenteimmobiliare.info (0.2%)   
6          fe26835492.91842100df.com (0.1%)   
7          c558b1f93b.d6468dab54.com (0.1%)   
8          7ea02e2ab2.e55a2001b9.com (0.1%)   
9                 eln.univ-batna2.dz (0.1%)   
10      secure-signals.permutive.app (0.1%)   
11         203783e0f1.43201617eb.com (0.1%)   
12                    odc.renault.dz (0.0%)   
13              secure.bali6nora.com (0.0%)   
14                   www.supradyn.dz (0.0%)   
15              adserver.autobip.com (0.0%)   
16                     www.condor.dz (0.0%)   
17                    www.mpt.gov.dz (0.0%)   
18                         www.ef.dz (0.0%)   
19                   www.ccomptes.dz (0.0%)   
20         cdn1-www.bureauveritas.dz (0.0%)   
21                      odc.dacia.dz (0.0%)   
22                 smetrics.bayer.dz (0.0%)   
23                        www.esi.dz (0.0%)   
24                     www.motrio.dz (0.0%)   
25               hydra.hotelhydra.dz (0.0%)   
26                   www.cdn.dz54.dz (0.0%)   
27                       www.dz54.dz (0.0%)   
28              www.albaraka-bank.dz (0.0%)   
29                     www.cnrdpa.dz (0.0%)   

                                            GERMANY  \
0                             www.google.de (15.2%)   
1                             script.ioam.de (0.7%)   
2   symplr-analytics-xddc4j3vva-ey.a.run.app (0.6%)   
3                              tm.ad-srv.net (0.6%)   
4                               pix.hyj.mobi (0.6%)   
5                               t.adcell.com (0.4%)   
6                                 benelph.de (0.4%)   
7                               l.ecn-ldr.de (0.4%)   
8                   ad13.adfarm1.adition.com (0.4%)   
9                             d.nativendo.de (0.3%)   
10                     tsvideo.saawsedge.com (0.3%)   
11                            de.hdtube.porn (0.3%)   
12                             p.gsitrix.com (0.3%)   
13                             o.gsitrix.com (0.3%)   

In [18]:

# Filter only rows with third-party tracking AND global category
df_tracking = summary_data[
    (summary_data['is_third_party_tracking'] == 1) &
    (summary_data['category'] == 'global')
].copy()

# Get list of countries (NOW consistent)
countries = df_tracking['location'].unique()

table_dict = {country: [] for country in countries}

for country in countries:
    df_country = df_tracking[df_tracking['location'] == country]
    df_other = df_tracking[df_tracking['location'] != country]

    # Collect all third-party origins in other countries
    origins_other = set()
    for s in df_other['distinct_third_party_tracking_origins']:
        if pd.isna(s):
            continue
        origins_other.update(o.strip() for o in s.split(';') if o.strip())

    # Collect origins unique to this country
    origins_country_only = []
    for s in df_country['distinct_third_party_tracking_origins']:
        if pd.isna(s):
            continue
        origins = set(o.strip() for o in s.split(';') if o.strip())
        for origin in origins:
            if origin not in origins_other:
                origins_country_only.append(origin)

    # Count frequency
    counter = Counter(origins_country_only)
    n_sites = len(df_country)

    table_dict[country] = [
        {
            "origin": origin,
            "pct_sites": count / n_sites * 100
        }
        for origin, count in counter.most_common(10)
    ]

# ---- Build DataFrame with padding ----
max_len = max(len(v) for v in table_dict.values())

df_result = pd.DataFrame({
    country: (
        [f"{e['origin']} ({e['pct_sites']:.1f}%)" for e in table_dict[country]]
        + [""] * (max_len - len(table_dict[country]))
    )
    for country in countries
})

df_result


GERMANY  \
0                              www.google.de (19.9%)   
1                       tsvideo.saawsedge.com (1.3%)   
2                              de.hdtube.porn (1.2%)   
3                   8841c61d83.372ae6b830.com (0.6%)   
4                   cb00f61c4d.50deb82526.com (0.3%)   
5                            ampcid.google.de (0.2%)   
6                   21404afb02.8b0e465f3e.com (0.2%)   
7  obgpm76tt0a0sgogzhdfe.redinuid.imrworldwide.co...   
8          cdn.track.production.webgains.team (0.2%)   
9                              de.xvideos.com (0.1%)   

                              INDIA                               UAE  \
0          www.google.co.in (21.5%)             www.google.ae (17.7%)   
1      go.stripchatgirls.com (0.8%)          display.yads.tech (1.6%)   
2  5a653527c0.1cd21cb1e4.com (0.5%)       ittr-asia.eskimi.com (0.4%)   
3  0997e86c03.c6073077cc.com (0.5%)  cad4defd63.a3c72512d5.com (0.2%)   
4        ampcid.google.co.in (0.3%)  c34352e2b2.8eabd9b030.com (0.1%)   
5         ads-g.juicyads.com (0.2%)           ampcid.google.ae (0.1%)   
6    apac1.smartadserver.com (0.2%)  7b1deec9a7.70de8b3bd7.com (0.1%)   
7  84d53ec4a2.c84c10fe3d.com (0.2%)              cdn2.teads.tv (0.1%)   
8  30dd07ff57.f680e0bdca.com (0.2%)         upc.udr.uol.com.br (0.1%)   
9   mboxedge41.tt.omtrdc.net (0.1%)        go.stripchat.webcam (0.1%)   

                                   ALGERIA  \
0                    www.google.dz (18.7%)   
1         dbb8b55e45.7345dc0372.com (0.3%)   
2           rtb-useast-v4.ezmob.com (0.2%)   
3                  ampcid.google.dz (0.2%)   
4  uvi-12796.agenteimmobiliare.info (0.2%)   
5         fe26835492.91842100df.com (0.2%)   
6         c558b1f93b.d6468dab54.com (0.2%)   
7         7ea02e2ab2.e55a2001b9.com (0.1%)   
8      secure-signals.permutive.app (0.1%)   
9         203783e0f1.43201617eb.com (0.1%)   

                                         USA  
0               pxl-us.tsyndicate.com (1.7%)  
1      visitor-risecode.omnitagjs.com (1.3%)  
2                       ad.tpmn.co.kr (1.3%)  
3                 ash.creativecdn.com (0.6%)  
4                adx-useast.adnxs.com (0.5%)  
5                        rc.rlcdn.com (0.3%)  
6  visitor.us-east4.gcp.omnitagjs.com (0.3%)  
7  rtb.ads.us-east.travelaudience.com (0.2%)  
8            nym2-ib.adnxs-simple.com (0.2%)  
9           932bd27cf7.6e64d84de7.com (0.2%)

# 6. Country specific organizations

In [19]:

org_locations = {}

# Step 1: Map organizations to the locations they appear in
for (cat, loc), group in summary_data.groupby(['category', 'location']):
    for domains_entry in group['distinct_third_party_tracking_domains']:
        if pd.isna(domains_entry):
            continue  # skip NaN
        
        # Convert string to list if needed
        if isinstance(domains_entry, str):
            domains_list = [d.strip() for d in domains_entry.split(';') if d.strip()]
        else:
            domains_list = list(domains_entry)  # assume iterable
        
        for domain in set(domains_list):  # count each domain once per site
            org = get_organization(domain)
            if org not in org_locations:
                org_locations[org] = set()
            org_locations[org].add(loc)

# Step 2: Filter organizations that appear in only one location
country_specific_orgs = {
    org: list(locs)[0] 
    for org, locs in org_locations.items() 
    if len(locs) == 1
}

# Step 3: Count percentage of sites per location
columns_data = {}

for loc in summary_data['location'].unique():
    group = summary_data[summary_data['location'] == loc]
    total_sites = group.shape[0]

    org_counts = Counter()
    for domains_entry in group['distinct_third_party_tracking_domains']:
        if pd.isna(domains_entry):
            continue

        # Convert string to list if needed
        if isinstance(domains_entry, str):
            domains_list = [d.strip() for d in domains_entry.split(';') if d.strip()]
        else:
            domains_list = list(domains_entry)

        # Get unique organizations for this site
        organizations = {get_organization(domain) for domain in domains_list}
        for org in organizations:
            if org in country_specific_orgs and country_specific_orgs[org] == loc:
                org_counts[org] += 1

    # Compute percentages
    org_percentages = {org: count / total_sites * 100 for org, count in org_counts.items()}

    # Sort descending
    sorted_orgs = sorted(org_percentages.items(), key=lambda x: x[1], reverse=True)

    # Format as "organization (xx%)"
    formatted_orgs = [f"{org} ({perc:.2f}%)" for org, perc in sorted_orgs]

    columns_data[loc] = formatted_orgs

# Step 4: Pad columns to equal length
max_len = max(len(orgs) for orgs in columns_data.values())
for col, orgs in columns_data.items():
    if len(orgs) < max_len:
        orgs.extend([''] * (max_len - len(orgs)))

# Step 5: Build DataFrame
df_country_specific_orgs = pd.DataFrame(columns_data)

df_country_specific_orgs.head(20)


UAE                  ALGERIA  \
0                        Gamned (0.02%)    Mallbazar.com (0.00%)   
1                       NoFraud (0.02%)       MyHeritage (0.00%)   
2         PRETTYLITTLETHING.COM (0.02%)  Thomson Reuters (0.00%)   
3   Abercrombie & Fitch Trading (0.02%)        WebTrends (0.00%)   
4                       Relay42 (0.01%)                            
5                     PressPage (0.01%)                            
6                 Paysafe Group (0.01%)                            
7                 Coherent Path (0.01%)                            
8                   Dubai Media (0.01%)                            
9                     TraceMyIP (0.01%)                            
10                 Cafe24 Corp. (0.01%)                            
11         Check Point Software (0.01%)                            
12                      Vivendi (0.01%)                            
13                      Caleres (0.01%)                            
14                   Threadloom (0.01%)                            
15      TD Bank Financial Group (0.01%)                            
16     MCN - Media Content News (0.01%)                            
17                       Lowe's (0.01%)                            
18                        Hyatt (0.01%)                            
19                      Blocket (0.01%)                            

                                    GERMANY  \
0                            adrule (0.42%)   
1                            GP One (0.20%)   
2                   United Internet (0.17%)   
3                      BurdaForward (0.15%)   
4           Sparkassen-Finanzportal (0.13%)   
5            Bezirksregierung Koeln (0.09%)   
6                          AdSpirit (0.08%)   
7                        Tabfoundry (0.07%)   
8   CBC Cologne Broadcasting Center (0.07%)   
9                       Psyma Group (0.06%)   
10                 Liquid New Media (0.06%)   
11                 Freistaat Bayern (0.05%)   
12             G+J Digital Products (0.05%)   
13         BerlinOnline Stadtportal (0.04%)   
14      Verlagsgesellschaft Madsack (0.04%)   
15             Universitaet Leipzig (0.03%)   
16               FUNKE Corporate IT (0.03%)   
17                  Auswärtiges Amt (0.03%)   
18         CHECK24 Vergleichsportal (0.03%)   
19         Mitteldeutscher Rundfunk (0.02%)   

                                            INDIA  \
0                      T.V. Today Network (0.04%)   
1                                Symptoma (0.02%)   
2            Vizury Interactive Solutions (0.02%)   
3              Novi Digital Entertainment (0.02%)   
4                           Bharti Airtel (0.02%)   
5              Clemens Schleiwies Company (0.02%)   
6                                   MyGov (0.02%)   
7   Guangzhou Banggood Network Technology (0.01%)   
8                             Morningstar (0.01%)   
9            Project Management Institute (0.01%)   
10                      First Derivatives (0.01%)   
11                               Experian (0.01%)   
12                     UnitedHealth Group (0.01%)   
13                     The Vanguard Group (0.01%)   
14                                 Eterno (0.01%)   
15                               apilayer (0.01%)   
16                                 Zeotap (0.01%)   
17                              Axis Bank (0.00%)   
18                       AO Kaspersky Lab (0.00%)   
19                     FREECHARGE PAYMENT (0.00%)   

                                      USA  
0                            TPMN (1.02%)  
1                        Optimize (0.08%)  
2                          Celtra (0.04%)  
3                       Hdfc Bank (0.02%)  
4   Internet Kholding E-generator (0.02%)  
5                         Gannett (0.02%)  
6              The Coryn Group II (0.02%)  
7                           Viqeo (0.01%)  
8                          Vdopia (0.01%)  
9                ETRADE Financial (0.01%)  
10                   

# 7. Prevalence of the GTM in the first party context

In [20]:
import re

def is_gtm(url):
    if pd.isna(url):
        return False
    url_lower = url.lower()
    return bool(re.search(r"gtm\.js|gtm\.start|googletagmanager", url_lower))


first_party_tracking_df['is_GTM'] = first_party_tracking_df['request_url'].apply(is_gtm)
first_party_tracking_df['is_GTM'].value_counts()


KeyError: 'request_url'

In [ ]:
# 1. Count total distinct eTLDs per category & location
total_sites = (
    first_party_tracking_df
    .groupby(['category', 'location'])['etld']
    .nunique()
    .reset_index(name='total_distinct_etlds')
)

# 2. Count distinct eTLDs with GTM (from previous step)
gtm_sites = (
    first_party_tracking_df[first_party_tracking_df['is_GTM']]
    .groupby(['category', 'location'])['etld']
    .nunique()
    .reset_index(name='distinct_etlds_with_GTM')
)

# 3. Merge totals with GTM counts
merged = pd.merge(gtm_sites, total_sites, on=['category', 'location'], how='right')

# 4. Fill NaN (if a group has zero GTM sites) with 0
merged['distinct_etlds_with_GTM'] = merged['distinct_etlds_with_GTM'].fillna(0)

# 5. Compute percentage
merged['percentage_GTM'] = (merged['distinct_etlds_with_GTM'] / merged['total_distinct_etlds']) * 100

# 6. Sort nicely
merged = merged.sort_values(['category', 'location']).reset_index(drop=True)

merged